In [1]:
from datetime import datetime, timedelta
import copy 

import download
import observatories
import analysis
import data
import correlation
import reference

%load_ext autoreload
%autoreload 2


In [2]:
_year = 2022
_month = 1
_day = 31 
_days = 1

nobg=True
bin_f=False
bin_t=False
flatten=True
bin_t_w=4
flatten_w=400
r_w=180
spec_range = [45, 81]

limit = 0.6

date_start = datetime(year=_year, month=_month, day=_day)
time_step = timedelta(days=1)
number_days = _days

observatory = [observatories.uni_graz, observatories.triest, observatories.swiss_landschlacht, observatories.oe3flb,
               observatories.alaska_haarp, observatories.alaska_cohoe, observatories.roswell, observatories.bir,
               observatories.indonesia, observatories.assa, observatories.swiss_muhen, observatories.swiss_hb9sct,
               observatories.egypt_alexandria, observatories.arecibo]

events_day = []
for i in range(number_days):
    date = date_start + time_step * i
    year = date.year
    month = date.month
    day = date.day
    download.downloadFullDay(year, month, day, observatory)
    stations = download.observatoriesAvailable(year, month, day)[1]
    sets = []
    for j in stations:
        sets.extend(data.listDataPointDay(year, month, day, j, spec_range))
    events = analysis.EventList([])
    for set1 in range(len(sets)):
        for set2 in range(set1 + 1, len(sets)):
            data1_raw = copy.deepcopy(sets[set1])
            data2_raw = copy.deepcopy(sets[set2])
            data1, data2 = data.fitTimeFrameDataSample(data1_raw, data2_raw)
            if data1 and data2:
                corr = correlation.Correlation(data1, data2, day, _no_background=nobg, _bin_freq=bin_f,
                                               _bin_time=bin_t, _flatten=flatten, _bin_time_width=bin_t_w,
                                               _flatten_window=flatten_w, _r_window=r_w)
                corr.calculatePeaks(_limit=limit)
                events += corr.peaks
            else:
                pass
    events.sort()
    events_day.append(events)
    #print(f"\n {date.year} {date.month} {date.day}")
    #print("mine")
    #print(events)
    #print("reference SWPC")
    #print(reference.referenceSWPC(year, month, day))
    #print("reference Monstein")
    #print(reference.referenceMonstein(year, month, day))
    #print("reference Monstein with 2 or more stations")
    #print(reference.referenceMonstein2orMore(year, month, day))
# return events_day

c:\Users\Lukas\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\window\rolling.py:321: UserWarning: Warning: converting a masked element to nan.
  values = ensure_float64(values)
f:\programming\eCallistoRadioBursts\data.py:234: UserWarning: Warning: converting a masked element to nan.
  arr = np.array(self.summedCurve)


In [3]:
r = 1
for i in events_day:
    print(r)
    r += 1
    for j in i:
        print(j, j.stations)
        print(observatories.ObservatorySet(j.stations).getSet())

1
['III', 00:05:21, 00:05:56, '0.8440'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
[' II', 00:13:06, 00:17:52, '0.8690'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
[' II', 00:24:49, 00:30:30, '0.8459'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
['III', 00:36:57, 00:37:26, '0.8108'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
[' II', 00:39:49, 00:46:12, '0.8862'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
['III', 00:51:51, 00:53:21, '0.9021'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
['III', 01:09:11, 01:10:34, '0.9345'] [ALASKA-COHOE, Australia-ASSA]
[[ALASKA-COHOE, Australia-ASSA]]
[' II', 02:31:10, 02:34:03, '0.9865'] [INDONESIA, Australia-ASSA]
[[INDONESIA, Australia-ASSA]]
['III', 03:06:33, 03:08:15, '0.8417'] [INDONESIA, Australia-ASSA]
[[INDONESIA, Australia-ASSA]]
['III', 04:39:44, 04:40:32, '0.8290'] [INDONESIA, Australia-ASSA]
[[INDONESIA, Australia-ASS

In [12]:
events_day_checked = []
for list_day in events_day:
    e_list = analysis.EventList([])
    for event in list_day:
        obs = observatories.ObservatorySet(event.stations)
        event_time = event.time_start
        event_time_ahead = event_time - timedelta(minutes=15)
        set_obs = obs.getSet()
        for i in set_obs:
            try:
                dp11 = data.createFromTime(event_time.year, event_time.month, event_time.day, str(event.time_start), i[0], spec_range)
                dp12 = data.createFromTime(event_time_ahead.year, event_time_ahead.month, event_time_ahead.day, str(event.time_start-timedelta(minutes=15)), i[0], spec_range)
                dp21 = data.createFromTime(event_time.year, event_time.month, event_time.day, str(event.time_start), i[1], spec_range)
                dp22 = data.createFromTime(event_time_ahead.year, event_time_ahead.month, event_time_ahead.day, str(event.time_start-timedelta(minutes=15)), i[1], spec_range)
                dp1, dp2 = data.fitTimeFrameDataSample([sum([dp11,dp12])], [sum([dp21,dp22])])
            except TypeError:
                continue
            except:                
                dp1 = data.createFromTime(event_time.year, event_time.month, event_time.day, str(event.time_start), i[0], spec_range)     
                dp2 = data.createFromTime(event_time.year, event_time.month, event_time.day, str(event.time_start), i[1], spec_range)       

            cor = correlation.Correlation(dp1, dp2, day,  
                                          _flatten=True, _bin_time=True, _bin_freq=True, _no_background=True,
                                          _r_window=30)
            cor.calculatePeaks()
            e_list += cor.peaks
    events_day_checked.append(e_list)

c:\Users\Lukas\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
f:\programming\eCallistoRadioBursts\data.py:201: UserWarning: Warning: converting a masked element to nan.
  self.spectrum_data.data = np.array(data_binned)
c:\Users\Lukas\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\lib\nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


AttributeError: 'list' object has no attribute 'spectrum_data'

In [27]:
events_day

[[['III', 03:25:40, 03:26:11, '0.6981'], ['III', 04:45:49, 04:46:17, '0.6629'], ['III', 10:03:26, 10:03:55, '0.6084'], ['III', 12:56:27, 12:57:26, '0.6408'], ['III', 13:42:49, 13:43:21, '0.9547'], ['III', 13:45:09, 13:45:23, '0.9627'], ['III', 14:52:10, 14:52:46, '0.6480'], ['III', 14:59:23, 14:59:59, '0.8087'], ['III', 15:14:35, 15:15:10, '0.6557'], ['III', 15:21:09, 15:21:17, '0.6010'], ['III', 17:39:53, 17:40:16, '0.6273'], ['III', 19:34:11, 19:34:56, '0.6519'], ['III', 20:06:31, 20:06:51, '0.6225'], ['III', 21:54:10, 21:54:51, '0.6626'], ['III', 21:58:13, 21:58:43, '0.6338'], ['III', 22:10:45, 22:11:33, '0.7522'], ['III', 22:24:36, 22:25:40, '0.6673'], ['III', 22:34:20, 22:34:31, '0.8388'], ['III', 22:58:06, 22:58:38, '0.6297']],
 [['III', 07:00:45, 07:01:15, '0.7153'], ['III', 07:46:12, 07:46:44, '0.7657'], ['III', 07:47:57, 07:48:13, '0.6416'], ['III', 08:53:50, 08:54:36, '0.7725'], ['III', 08:58:21, 08:58:54, '0.7235'], ['III', 09:00:22, 09:00:42, '0.6148'], ['III', 09:01:53, 09

In [31]:
event

['III', 10:42:11, 10:43:02, '0.7665']

In [5]:
events_day_checked = []
for list_day in events_day:
    e_list = analysis.EventList([])
    for event in list_day:
        obs = observatories.ObservatorySet(event.stations)
        year = event.time_start.year
        month = event.time_start.month
        day = event.time_start.day
        set_obs = obs.getSet()
        for i in set_obs:
            dp1 = data.createFromTime(year, month, day, str(event.time_start), i[0], spec_range)      # needs to search for the file before this one (if close to start fo file)  [ - timedelta(minutes=15)]
            dp2 = data.createFromTime(year, month, day, str(event.time_start), i[1], spec_range)            # same here  , also data.fitTimeFrameDataSample(), because reasons
            cor = correlation.Correlation(dp1, dp2, day,  
                                          _flatten=True, _bin_time=True, _bin_freq=True, _no_background=True,
                                          _r_window=30)
            cor.calculatePeaks()
            e_list += cor.peaks
    events_day_checked.append(e_list)

TypeError: buffer is too small for requested array

In [ ]:
events_day

In [ ]:
events_day_checked

In [ ]:
# _days = 28
ref_swpc = [reference.referenceSWPC(year, month, i) for i in range(_day, _day + _days)]
ref_monstein = [reference.referenceMonstein(year, month, i) for i in range(_day, _day + _days)]
ref_monstein2p = [reference.referenceMonstein2orMore(year, month, i) for i in range(_day, _day + _days)]
ref_swpc_strong = [analysis.EventList([j for j in i if j.burst_type == "III/2" or j.burst_type == "III/3"]) for i in ref_swpc]
# events with burst_type.startswith(III/) or (II/)  ??

events_num = sum(len(i) for i in events_day[:_days])

events_swpc = sum(len(i)for i in ref_swpc[:_days])
events_swpc_strong = sum(len(i)for i in ref_swpc_strong[:_days])

events_monstein = sum(len(i)for i in ref_monstein[:_days])
events_monstein2p = sum(len(i)for i in ref_monstein2p[:_days])

false_positives_all = []
missed_strong_all = []
missed_swpc_all = []
missed_monstein_all = []
missed_monstein2p_all = []
for i in range(_day, _day + _days):
    false_positives = (events_day[i-1] - ref_swpc[i-1]) - ref_monstein[i-1]
    false_positives_all.extend(false_positives)
    print(f"\n{year} {month} {i}\nfalse positives: ", false_positives)

    missed_swpc = ref_swpc[i-1] - events_day[i-1]
    print(f"missed swpc:", missed_swpc)
    missed_swpc_all.extend(missed_swpc)

    missed_monstein = ref_monstein[i-1] - events_day[i-1]
    print(f"missed monstein:", missed_monstein)
    missed_monstein_all.extend(missed_monstein)

    missed_monstein2p = ref_monstein2p[i-1] - events_day[i-1]
    print(f"missed monstein measured at 2+ of my stations:", missed_monstein2p)
    missed_monstein2p_all.extend(missed_monstein2p)

    missed_strong = ref_swpc_strong[i-1] - events_day[i-1]
    print(f"missed strong:", missed_strong)
    missed_strong_all.extend(missed_strong)

print(f"\n------------------------------\nEvents found: {events_num}")
print(f"False positives: {len(false_positives_all)} ({len(false_positives_all)/events_num*100}%)")
print(f"\nFailed bursts SWPC: {len(missed_swpc_all)} ({len(missed_swpc_all)/events_swpc*100}%)")
print(f"Failed bursts SWPC (strong): {len(missed_strong_all)} ({len(missed_strong_all)/events_swpc_strong*100}%)")
print(f"Failed bursts Monstein: {len(missed_monstein_all)} ({len(missed_monstein_all)/events_monstein*100}%)")
print(f"Failed bursts Monstein measured at >=2 stations: {len(missed_monstein2p_all)} ({len(missed_monstein2p_all)/events_monstein2p*100}%)")

In [ ]:
events_day